# Hedgefund careers

In [1]:
import sel_scrape as sc
import pandas as pd
import datetime
from IPython.display import display, HTML
import re
import pathlib
from xml.etree import  cElementTree as ET
import pdb
import traceback
import time


## Blackrock

In [2]:
DICT_XPATH = {
    'title':"//li[@class='job-data title']",
    'href':"//li[@class='job-data title']/div/a",
    'locs':"//li[@class='job-data store_id']",
    'cats':"//li[@class='job-data parent_category']",
    'posted_date':"//li[@class='job-data compliment']",
    'read_more':"//a[@class='black-link read-more']",
    'full_description':"//div[@data-field='responsibilities']",
    'req':"//b[contains(text(),'requisition')]/ancestor::p[1]/following-sibling::p"
}
MAIN_URL ="https://careers.blackrock.com/job-search-results/"
HOME_FOLDER = str(pathlib.Path.home())

In [3]:
scc = sc.SelScrape(headless=False)
sda = sc.SelDictAccess(main_url=MAIN_URL,dict_xpath=DICT_XPATH,sac=scc)

In [ ]:
df = None
for i in range(1,100):
    print(f'procesing page {i}')
    try:
        sda.sac.goto(f"https://careers.blackrock.com/job-search-results?pg={i}")
        test_element = sda.find_xpath('title')
        if test_element['status'] is not None:
            break
        d = {
            'title':[e.text for e in sda.find_xpath('title')['value']],
            'job_url':[e.get_attribute('href') for e in sda.find_xpath('href')['value']],
            'location':[e.text for e in sda.find_xpath('locs')['value']],
            'cat':[e.text for e in sda.find_xpath('cats')['value']],
            'posted_date':[e.text for e in sda.find_xpath('posted_date')['value']],
        }
        df_temp = pd.DataFrame(d)
        if df is None:
            df = df_temp.copy()
        else:
            df = df.append(df_temp,ignore_index=True)
            df.index = list(range(len(df)))
    except Exception as e:
        traceback.print_exc()
# df = df.rename(columns={'loc':'location'})
# df = df.rename(columns={'href':'job_url'})
df['job_num'] = df.job_url.apply(lambda s:re.findall('job\/[0-9]{5,12}\/',s)[0].replace('/','').replace('job',''))
df_black_rock_urls = df.copy()


In [ ]:
def scroll_shim(passed_in_driver, object):
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % (
        x,
        y
    )
    scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)

In [ ]:
def click_read_more(ind):
    driver = sda.sac.driver
    sda.sac.goto(df.loc[ind].job_url)
    source_element = sda.find_xpath('read_more')['value'][0]
    if 'firefox' in driver.capabilities['browserName']:
        scroll_shim(driver, source_element)
    # scroll_shim is just scrolling it into view, you still need to hover over it to click using an action chain.
    actions = sc.webdriver.ActionChains(driver)
    actions.move_to_element(source_element)
    actions.click()
    actions.perform()


In [ ]:
descriptions = []
reqs = []
inds = []


In [ ]:
index_start = 0
for ind in [k for k in df.index if k>index_start]: 
    req = 0
    description = None
    try:
        click_read_more(ind)
        time.sleep(1)
        description_element_list = sda.find_xpath('full_description')['value']
        description = description_element_list[1].text
        req = sda.find_xpath('req')['value'][0].text
    except Exception as e2:
        traceback.print_exc()    
    sda.logger.info(f'finished index {ind}. req = {req}')
                
    descriptions.append(description)
    reqs.append(req)
    inds.append(ind)
    

In [ ]:
df_blackrock_descriptions = pd.DataFrame({'req':reqs,'description':descriptions},index=inds)


In [ ]:
df_blackrock = df_black_rock_urls.join(df_blackrock_descriptions)

In [ ]:
df_blackrock.iloc[101]

## AQR

In [ ]:
main_url = 'https://boards.greenhouse.io/aqr'
ss = sc.SelScrape(headless=False)

In [ ]:
ss.goto(main_url)

In [ ]:
jobs[2].get_attribute('href')

In [ ]:
jobs = ss.findxpath('//div[@class="opening"]/a')['value']
locs = ss.findxpath('//div[@class="opening"]/span[@class="location"]')['value']
d = {
    'title':[j.text for j in jobs ],
    'href':[j.get_attribute('href') for j in jobs ],
    'location':[j.text for j in locs],
}
df_aqr = pd.DataFrame(d)
df_aqr

In [ ]:
ss.goto(df_aqr.iloc[2].href)

In [ ]:
c = 'contains(text(),"What") and contains(text(),"Bring")'
# req_elements = ss.findxpath(f'//div[@id="content"]//strong[{c}]/parent::p/following-sibling::ul/li')['value']
#[e.text for e in req_elements]
ss.findxpath(f'//div[@id="content"]//strong[{c}]/parent::p/following-sibling::ul/li')


In [ ]:
lines = ss.findxpath('//div[@id="content"]')['value'][0].text.split('\n')
lines_dict = {'team':lines[1],'role':lines[3],'experience':lines[5:]}
lines

In [ ]:
c = 'contains(text(),"What") and contains(text(),"Bring")'
descriptions = []
for i in range(len(df_aqr)):
    r = df_aqr.iloc[i]
    ss.goto(r.href)
    lines = ss.findxpath('//div[@id="content"]')['value'][0].get_property('innerHTML')#.text.split('\n')
#     req_elements = ss.findxpath(f'//div[@id="content"]//strong[{c}]/parent::p/following-sibling::ul/li')['value']
#     experiences.append([e.text for e in req_elements])
#     experiences.append(lines[5:])
    descriptions.append(lines)
df_aqr_2 = df_aqr.copy()
# df_aqr_2['team'] = teams
# df_aqr_2['role'] = roles
df_aqr_2['description'] = descriptions


In [ ]:
ss.goto(df_aqr.iloc[1].href)
l = ss.findxpath('//div[@id="content"]')['value'][0].get_property('innerHTML')

In [ ]:
df_aqr_2_greenwich.columns.values

In [ ]:
df_aqr_2_greenwich = df_aqr_2[df_aqr_2.location.str.contains('Greenwich')]
for i in range(len(df_aqr_2_greenwich)):
    r = df_aqr_2_greenwich.iloc[i]
    print(f'**************** {r.location} {r.title} ********************')
    display(HTML(r.description))
#     for d in r.description:
#         print(d)
    print('')
    

## Verizon

In [ ]:
ss = sc.SelScrape(headless=False)

### get all urls

In [ ]:
df_verizon = None
for i in range(1,200):
    print(f'processing page {i}')
    try:
        main_url = f"https://www.verizon.com/about/work/search/jobs?page={i}&per_page=100&sort_by=cfml10%2Cdesc"
        ss.goto(main_url)
        h = ss.findxpath('//table')['value'][0].get_attribute('innerHTML').replace('\n',' ')
        df_temp = pd.read_html('<table>' + h + '</table>')[0]
        urls = [e.get_attribute('href') for e in ss.findxpath('//td[@class="jobs_table_item_title"]/a')['value']]
        df_temp['url'] = urls
        if df_verizon is None: 
            df_verizon = df_temp.copy()
        else:
            df_verizon = df_verizon.append(df_temp,ignore_index=True)
            df_verizon.index = list(range(len(df_verizon)))
    except Exception as e:
        print(f'{str(e)}')
        break
df_verizon

### Get subset of tech from specific states

In [ ]:
state_list = ['FL','NJ','NY','CT']
loc_list = [s for s in df_verizon.Location.unique() if any(sub in s for sub in state_list)]
df_verizon_2 = df_verizon[(df_verizon.Location.isin(loc_list))]
df_verizon_2_tech = df_verizon_2[df_verizon_2.Category.str.lower().str.contains('technology')]
df_verizon_2_tech 

In [ ]:
main_text = '//div[@class="cs_item_text"]'
def get_page(u):
    ss.goto(u)
    try:
        r = ss.findxpath(main_text)
        if r['status'] is not None:
            return r['status']
        h = r['value'][0].get_property('innerHTML')
        return (h)
        
    except Exception as e:
        return str(e)

def get_html_list(df):
    html_list = []
    for i in range(len(df)):
        r = df.iloc[i]
        print(f'****** page {i}: {r.Title} {r.Location} ******')
        try:        
            p = get_page(r.url)
            html_list.append(p)            
            display(HTML(p))
            print('')
        except Exception as e:
            print(str(e))
            html_list.append(str(e))
    return html_list

In [ ]:
html_list = []
for i in range(len(df_verizon_2_tech)):
    r = df_verizon_2_tech.iloc[i]
    try:        
        p = get_page(r.url)
        html_list.append(p)
        print(f'******************* page {i}: {r.Title} {r.Location} ***********************')
        display(HTML(p))
        print('')
    except Exception as e:
        print(str(e))
df_verizon_2_tech['description'] = html_list      

In [ ]:
df_verizon_2_tech_java = df_verizon_2_tech[df_verizon_2_tech.Title.str.lower().str.contains('java')]
for i in range(len(df_verizon_2_tech_java)):
    r = df_verizon_2_tech_java.iloc[i]
    p = r.description
    html_list.append(p)
    print(f'******** page {i}: {r.Title} {r.Location} ***********')
    display(HTML(p))
    print('')

In [ ]:
df_verizon_python = df_verizon[df_verizon.Title.str.lower().str.contains('python')]
df_verizon_python['description'] = get_html_list(df_verizon_python)

## Small funds

In [ ]:
ss = sc.SelScrape(headless=False)
main_url = "https://www.holdingschannel.com/all/"

In [ ]:
ss.goto(main_url)

In [ ]:
a_to_z = ss.findxpath('//a[contains(@href,"letter=")]')['value']
a_to_z_hrefs = sorted(list(set([e.get_attribute('href') for e in a_to_z])))


In [ ]:
fund_list_xpath = '//h1[contains(text(),"Alphabetical")]/following-sibling::a[contains(@href,"www.holdingschannel.com")]'
all_funds_hrefs = []
all_funds_aums_this_year = []
all_funds_aums_last_year = []
for a_z_h in a_to_z_hrefs[:2]:
    print(a_z_h)
    ss.goto(a_z_h)
    fund_element_list = ss.findxpath(fund_list_xpath)['value']
    fund_href_list = [e.get_attribute('href') for e in fund_element_list]
    for fund_href in fund_href_list[:2]: 
        try:
            all_funds_hrefs.append(fund_href)
            ss.goto(fund_href)        
    #         [e.get_property('innerHTML') for e in ss.findxpath('//')['value']]
            v = ss.findxpath('//td[@class="menuoff"]/parent::tr/following-sibling::tr/td')['value'][0].get_property('innerHTML')
            aum_list = re.findall(pattern='At[ ]{1,3}[0-1][0-9]/[[0-3][0-9]/20[1-3][0-9][:].{1,4}[0-9,]{1,}',string=v)
            aum_values = [re.findall(pattern='\$[0-9,]{1,}$',string=a)[0].replace('$','').replace(',','') for a in aum_list]
            aum_values = [float(str(s)) for s in aum_values]
            all_funds_aums_this_year.append(aum_values[0])
            all_funds_aums_last_year.append(aum_values[1])
        except Exception as e:
            print(f'Exception on {fund_href}')
            print(f'Exception:{str(e)}')
        
d = {'href':all_funds_hrefs,'aum_this_year':all_funds_aums_this_year,'aum_last_year':all_funds_aums_last_year}
df_all_funds = pd.DataFrame(d)
    

In [ ]:
a_to_z_hrefs

In [ ]:
a_to_z_hrefs

In [ ]:
fname = HOME_FOLDER + '/downloads/IA_FIRM_SEC_Feed_05_17_2019.xml'
tree = ET.ElementTree(file=fname)


In [ ]:
ee =tree.findall('.//Firm/MainAddr')[0]
ee.attrib['Strt1']

In [ ]:
firms = tree.findall('.//Firm')
legal_names = [e.attrib['LegalNm'] for e in tree.findall('.//Firm/Info')]
web_sites = [e.findall('.//WebAddrs/WebAddr')[0].text if len(e.findall('.//WebAddrs/WebAddr'))>0 else None for e in firms]
def get_address_from_firm(e):
    s1 =  '' if 'Strt1' not in e.attrib else e.attrib['Strt1']
    s2 = '' if 'Strt2' not in e.attrib else e.attrib['Strt2']
    city = '' if 'City' not in e.attrib else e.attrib['City']
    state = '' if 'State' not in e.attrib else e.attrib['State']
    return ' '.join([s1,s2,city,state])
addresses = [get_address_from_firm(e) for e in tree.findall('.//Firm/MainAddr')]
aums = [-1 if 'Q5DF3' not in e.attrib else int(e.attrib['Q5DF3']) for e in tree.findall('.//Firm/FormInfo/Part1A/Item5D')]


In [ ]:
df_funds = pd.DataFrame({'legal_name':legal_names,'web_site':web_sites,'addresse':addresses,'aum':aums})

In [ ]:
df_small_funds = df_funds[(df_funds.aum>100) & (df_funds.aum<100000000)]
df_small_funds

In [ ]:
df_small_funds_https = df_small_funds[['https' in str(s) for s in df_small_funds.web_site.str.lower()]]
df_small_funds_https

In [ ]:
ss = sc.SelScrape(headless=False)
ss.goto(df_small_funds_https.iloc[0].web_site)

In [ ]:
career_elements = []
hrefs = []
for s in df_small_funds_https.web_site:
    try:
        ss.goto(s)
        c = ss.findxpath('//*[contains(translate(text(),"CAREER","career"),"career")]')
        if c['value'] is not None:
            print(s)
            career_elements.append(c['value'][0])
            hrefs.append(s)
    except Exception as e:
        print(str(e))

In [ ]:
df_small_funds_https

In [ ]:
url = 'https://medium.com/@InvestorDeck/hacking-investment-research-the-best-free-sites-to-research-stocks-37ff814adf68'
ss  = sc.SelScrape(headless=False)
ss.goto(url)


In [ ]:
h = '//p[contains(@class,"raf-after--h3")]/a'
[e.text for e in ss.findxpath(h)['value']]

### Chase careers


In [4]:
import sys,os
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.abspath('.'))
if os.path.abspath('..') not in sys.path:
    sys.path.append(os.path.abspath('..'))
from selscrape import chase_careers as cc

In [3]:
if __name__ == '__main__':
    chase = cc.Chase()
    df = chase.get_jobs()
    df.to_csv('./temp_folder/chase_jobs.csv',index=False, encoding = 'utf-8')


processed page: 0  num jobs: 30
processed page: 1  num jobs: 60
processed page: 2  num jobs: 90
processed page: 3  num jobs: 120
processed page: 4  num jobs: 150
processed page: 5  num jobs: 180
processed page: 6  num jobs: 210
processed page: 7  num jobs: 240
processed page: 8  num jobs: 270
processed page: 9  num jobs: 300
processed page: 10  num jobs: 330
processed page: 11  num jobs: 360
processed page: 12  num jobs: 390
processed page: 13  num jobs: 420
ftotal for https://jobs.chase.com/ListJobs/All/Country-US/State-NY/City-New-York/: 420
processed page: 0  num jobs: 450
processed page: 1  num jobs: 480
processed page: 2  num jobs: 489
ftotal for https://jobs.chase.com/ListJobs/All/Country-US/State-NJ/City-Jersey-City/: 489


In [5]:
df_chase = pd.read_csv('temp_folder/chase_jobs.csv')

In [6]:
def containsit(df,value_list,ignore_list=None):
    df_current = df.copy()
    for value in value_list:
        df_current = df_current[df_current.description.str.lower().str.contains(value)]
        if len(df_current)<1:
            break
    il = [] if ignore_list is None else ignore_list
    for value in il:
        df_current = df_current[~df_current.description.str.lower().str.contains(value)]
        if len(df_current)<1:
            break
    return df_current

In [7]:
df_chase_python = containsit(df_chase,'associate python'.split(),ignore_list=['ph.d','phd'])
# df_chase_python = containsit(df_chase,'ph.d python'.split())

In [8]:
len(df_chase_python),df_chase_python.req.unique()

(19, array(['Req #: 190091231', 'Req #: 190059389', 'Req #: 190086691',
        'Req #: 190078025', 'Req #: 190087274', 'Req #: 190063171_1',
        'Req #: 190085718', 'Req #: 180112375', 'Req #: 190033468',
        'Req #: 180112376', 'Req #: 190068501', 'Req #: 190054990',
        'Req #: 190085724', 'Req #: 190057453', 'Req #: 190073138',
        'Req #: 190073144', 'Req #: 190073136', 'Req #: 190085563',
        'Req #: 190080193'], dtype=object))

In [9]:
for req in df_chase_python.req.unique():
    dft = df_chase_python[df_chase_python.req==req]
    print(dft.iloc[0].title)
    print(dft.iloc[0].description)
    print("*******************************************************")
    print(" ")

Commercial Banking - CCBSI Reporting & Analytics, Finance & Business Management - Associate
Commercial Banking - CCBSI Reporting & Analytics, Finance & Business Management - Associate
Req #: 190091231
Location: New York, NY, US
Job Category: Accounting/Finance/Audit/Risk
Job Description:
The Corporate Client Banking & Specialized Industries (CCBSI) group is a corporate banking organization focused on companies typically with revenues of over $500mm which require traditional banking services and investment banking products. The group partners with investment bankers, credit and other internal product partners to offer customized financing solutions to our corporate clients. Our competitive advantage is that we cover our markets in a truly unique way – our corporate bankers and investment banking partners work together to build relationships with our clients and offer tailored solutions, ranging from cash management to raising capital and structuring complex financial products. Our inter

### Bridgewater Associates

In [10]:
df_bridge = pd.read_csv('temp_folder/bridgewater_jobs.csv')

In [13]:
for desc in df_bridge[df_bridge.description.str.lower().str.contains("python")].description:
    print(desc)

  Data Engineer – Core Management
Bridgewater Associates, LP
Bridgewater Associates is focused on understanding how the world works.  By having the deepest possible understanding of the global economy and financial markets, and translating that understanding into great portfolios and strategic partnerships with institutional clients, we’ve built a distinct track record of success.  We’ve done this for more than 40 years by having great people operate in a culture of radical truth and radical transparency. Today, we manage about $160 billion for approximately 350 of the largest and most sophisticated global institutional clients including public and corporate pension funds, university endowments, charitable foundations, supranational agencies, sovereign wealth funds, and central banks. 
Department Overview:
Core Management’s goal is to ensure management throughout Bridgewater is excellent (consistent with Bridgewater’s culture and Principles), both now and in perpetuity. The department 